<br>
<br>
# **금융 데이터 정리 및 Maria DB로 전환**
2018-06-16 
1. 우선 Sqlite3 를 CSV 등으로 추출정리 한다
1. 이를 Maria DB 등의 대용량 저장DB로 변환한다

<br><br>
## **1 New DataSet**
시장에서 새로운 정보를 수집하기

<br>
### **01 stock code 를 새롭게 수집하기**
> 기업이름 Yahoo, Google, KRX Code 정리

In [1]:
from googlefinance.get import get_code
codes         = get_code()
codes.columns = ['Google', 'Code','Name']

from data.fin import sqlite_new_code
codes         = sqlite_new_code(codes)
codes.to_csv('./data/codes.csv', index=None, encoding='ms949')
codes.head()

We can get the Code Numbers of "NYSE", "NASDAQ" and "KRX(default)"
Crawling the Kospi & Kosdaq (default) 's codes


,Code,Code_int,Google,Name
0,08537M,0,KOSDAQ:08537M,루트로닉3우C
1,00680K,0,KRX:00680K,미래에셋대우2우B
2,00088K,0,KRX:00088K,한화3우B
3,02826K,0,KRX:02826K,삼성물산우B
4,03473K,0,KRX:03473K,SK우


<br><br>
## **2 기존에 저장된 Sqlite3 전처리**
기본적인 SQL 의 CRUD 명령과 해당 모듈을 정리해 나아가기
1. 현재 DB가 1GB를 넘어가므로 필터링해서 추출 후 CSV로 백업하기
1. 추후 CSV로 분할된 정보를 MariaDB 나 Psql 등에 저장 (Maria DB가 우선 가벼워서 활용도가 높다)

<br>
### **01 Table 목록 추출**
**FROM** sqlite_master 에서 추출

In [2]:
from data.fin import sqlite_split_to_csv
sqlite_split_to_csv('./data/krxStock.db')

stock_volume is saved to CSV
stock_trader is saved to CSV
stock_change is saved to CSV
stock_trader_top is saved to CSV
stock_codes is saved to CSV
stock_price is saved to CSV
CSV saved is Done..


<br>
### **02 stock 테이블 정렬 및 코드명 전처리**
> 2018-04-11 부터 코드가 float() 형식으로 저장, 이를 정리할 필요가 있다

In [3]:
# 거래원별 데이터 전처리
from data.fin import sqlite_fix_code
url = './data/stock_trader.csv'
df  = sqlite_fix_code(url)
df  = df.reset_index()
df.to_csv(url, index=None)
del df

import pandas as pd
df  = pd.read_csv(url, dtype={'Code': str})
df.head(3)

,Date,Code,개인,외국인계,기관계,증권,투신,은행,종금,보험,기금,기타
0,2016-05-18,047770,0,0,0,0,0,0,0,0,0,0
1,2016-05-18,014200,4678,-756,0,0,0,0,0,0,0,-5100
2,2016-05-18,009780,-2124,0,0,0,0,0,0,0,0,0


In [4]:
# 종목별 가격 전처리
url = './data/stock_price.csv'
df = sqlite_fix_code(url)
df = df.reset_index()
df.to_csv(url, index=None)
del df

import pandas as pd
df = pd.read_csv(url, dtype={'Code': str})
df.head(3)

,Date,Code,Price,High,Low,Volume
0,2000-01-04,023150,660,700,612,25000
1,2000-01-04,009450,2500,2500,2290,61500
2,2000-01-04,015350,15900,15950,14900,26700


In [5]:
# 상위 거래원 전처리
from data.fin import sqlite_fix_code
url       = './data/stock_trader_top.csv'
df        = sqlite_fix_code(url, value_to_int=False)
# 상위거래원 이름에 공백으로 중복표현을 단일화
df.Trader = df.Trader.apply(lambda x : x.replace(' ', ''))
# '전체'라는 거래원 내용을 삭제하기
err_index = df[df.Trader == '전체'].index
err_index = list(err_index)
df        = df.drop(index = err_index)
df        = df.reset_index()
df.to_csv(url, index=None)
del df

import pandas as pd
df = pd.read_csv(url, dtype={'Code': str})
df.tail(3)

,Date,Code,Trader,Net,Buy,Ask
10850693,2018-06-15,238490,이베스트,-222,278,500
10850694,2018-06-15,238490,NH투자증권,-82,0,82
10850695,2018-06-15,238490,삼성,-10,0,10


<br><br>
## **2 DataBase 를 RDB로 바꾸기**
1. 우선은 순수한 ROW Table DATA를 생성하는데 목적을 두자
1. RDB 변경 작업은 DataBase 내부에서 처리 하도록 하자
1. 변경작업은 부분처리용 **_Instance_** 를 써서 반복을 최소화 한다

<br>
### **01 Table 원본 추출**
**FROM** sqlite_master 에서 추출

In [15]:
import pandas as pd
url = './data/stock_price.csv'
df = pd.read_csv(url, dtype={'Code': str})
df.tail(3)

,Date,Code,Price,High,Low,Volume
5936297,2018-06-15,115160,8830,9040,8830,84520
5936298,2018-06-15,200670,33100,33450,32500,27854
5936299,2018-06-15,238490,14300,14500,14200,11125


In [16]:
codes = pd.read_csv('./data/codes.csv', encoding='ms949', dtype={'Code':str})
codes.tail(3)

,Code,Code_int,Google,Name
2311,002785,2785,KRX:002785,진흥기업우B
2312,001529,1529,KRX:001529,동양3우B
2313,002787,2787,KRX:002787,진흥기업2우B


In [17]:
# 제대로 된 코드와 문제있는 코드 구분
from data.fin import sqlite_split_code
code_check           = list(set(df.Code))
code_ok, code_errors = sqlite_split_code(codes, code_check)
print("Possible company is : {}\nNot possible is {}".format(len(code_ok), len(code_errors)))

Possible company is : 2050
Not possible is 19


In [18]:
# # 오류로 분류된 기업들이 아직 남아있는지를 확인한다
# from data.fin import sqlite_check_code
# sqlite_check_code(code_errors)

In [19]:
code_errors

['221950',
 '217810',
 '225440',
 '223040',
 '221200',
 '215750',
 '039790',
 '087220',
 '900180',
 '219580',
 '219860',
 '099830',
 '218710',
 '226850',
 '008000',
 '219960',
 '215580',
 '051170',
 '222390']

<br>
### **03 거래원별  Foreign Key를 추가하기**
거래원별 index를 추가

In [10]:
# 새롭게 추가되고 줄어든 거래원을 어떻게 처리할 것인가?
# 무엇보다 이를 외래키로 작업해도 용량이 별로 줄어들지 않았다 !! ???
# 우선은 주석으로 처리후 추후에 생각해보자

# # df_trader : 거래원 Trader Series 생성
# # 생성한 Series를 사용하여 Foreign Key로 추가한다
# df_trader = list(set(df.Trader))
# df_trader = pd.Series(df_trader).sort_values().reset_index(drop=True)
# print(len(df_trader))

# for trader in df_trader:
#     # 거래원의 인덱스 주소 추출
#     idx = df_trader[df_trader == trader].index[0]

#     # 원본 데이터프레임에서 해당 인덱스만 추출
#     idx_org = list(df[df.Trader == trader].index)
#     df.Trader.iloc[idx_org] = int(idx)

# print('Done')

In [11]:
# for cod in codes.Code:
#     # 거래원의 인덱스 주소 추출
#     idx = codes[codes.Code == cod].index[0]

#     # 원본 데이터프레임에서 해당 인덱스만 추출
#     idx_org = list(df[df.Code == cod].index)
#     df.Code.iloc[idx_org] = int(idx)

# print('Done')